In [1]:
import sqlite3
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import numpy as np

Inner Join on Chinook Database

In [2]:
conn = sqlite3.connect('chinook.db')
query = """
SELECT c.CustomerId, c.FirstName, c.LastName, COUNT(i.InvoiceId) AS TotalInvoices
FROM customers c
INNER JOIN invoices i ON c.CustomerId = i.CustomerId
GROUP BY c.CustomerId, c.FirstName, c.LastName
ORDER BY TotalInvoices DESC;
"""
result_df = pd.read_sql_query(query, conn)
print(result_df)
conn.close()

    CustomerId  FirstName      LastName  TotalInvoices
0            1       Luís     Gonçalves              7
1            2     Leonie        Köhler              7
2            3   François      Tremblay              7
3            4      Bjørn        Hansen              7
4            5  František   Wichterlová              7
5            6     Helena          Holý              7
6            7     Astrid        Gruber              7
7            8       Daan       Peeters              7
8            9       Kara       Nielsen              7
9           10    Eduardo       Martins              7
10          11  Alexandre         Rocha              7
11          12    Roberto       Almeida              7
12          13   Fernanda         Ramos              7
13          14       Mark       Philips              7
14          15   Jennifer      Peterson              7
15          16      Frank        Harris              7
16          17       Jack         Smith              7
17        

Outer Join on Movie Data

In [3]:
movies_df = pd.read_csv('movie.csv')
df_director_color = movies_df[['director_name', 'color']]
df_director_reviews = movies_df[['director_name', 'num_critic_for_reviews']]
left_join_df = pd.merge(df_director_color, df_director_reviews, on='director_name', how='left')
outer_join_df = pd.merge(df_director_color, df_director_reviews, on='director_name', how='outer')
left_join_count = len(left_join_df)
outer_join_count = len(outer_join_df)
print(f"Number of rows in left join result: {left_join_count}")
print(f"Number of rows in full outer join result: {outer_join_count}")

Number of rows in left join result: 30300
Number of rows in full outer join result: 30300


Grouped Aggregations on Titanic

In [4]:
titanic_df = pd.read_excel('titanic.xlsx')
grouped_titanic = titanic_df.groupby('Pclass').agg(
    Average_Age=('Age', 'mean'),
    Total_Fare=('Fare', 'sum'),
    Passenger_Count=('PassengerId', 'count')
).reset_index()

print(grouped_titanic)

   Pclass  Average_Age  Total_Fare  Passenger_Count
0       1    38.233441  18177.4125              216
1       2    29.877630   3801.8417              184
2       3    25.140620   6714.6951              491


Multi-level Grouping on Movie Data

In [5]:
movies_df = pd.read_csv('movie.csv')
grouped_movies = movies_df.groupby(['color', 'director_name']).agg(
    Total_Critic_Reviews=('num_critic_for_reviews', 'sum'),
    Average_Duration=('duration', 'mean')
).reset_index()

print(grouped_movies)

                color       director_name  Total_Critic_Reviews  \
0     Black and White      Akira Kurosawa                 153.0   
1     Black and White      Aleksey German                 121.0   
2     Black and White        Alex Garland                 489.0   
3     Black and White     Alexander Payne                 433.0   
4     Black and White    Alfred Hitchcock                 434.0   
...               ...                 ...                   ...   
2485            Color       Zoran Lisinac                  17.0   
2486            Color  Álex de la Iglesia                  71.0   
2487            Color    Émile Gaudreault                  67.0   
2488            Color        Éric Tessier                   9.0   
2489            Color       Étienne Faure                   9.0   

      Average_Duration  
0                202.0  
1                177.0  
2                108.0  
3                115.0  
4                119.0  
...                ...  
2485             108

Apply a Custom Function on Titanic

In [6]:
def classify_age(age):
    if age < 18:
        return 'Child'
    else:
        return 'Adult'

titanic_df['Age_Group'] = titanic_df['Age'].apply(classify_age)
print(titanic_df[['Age', 'Age_Group']].head())

    Age Age_Group
0  22.0     Adult
1  38.0     Adult
2  26.0     Adult
3  35.0     Adult
4  35.0     Adult


Normalize Employee Salaries

In [7]:
employee_df = pd.read_csv('employee.csv')
employee_df['Normalized_Salary'] = employee_df.groupby('DEPARTMENT')['BASE_SALARY'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min())
)

print(employee_df.head())

   UNIQUE_ID               POSITION_TITLE                     DEPARTMENT  \
0          0  ASSISTANT DIRECTOR (EX LVL)    Municipal Courts Department   
1          1            LIBRARY ASSISTANT                        Library   
2          2               POLICE OFFICER  Houston Police Department-HPD   
3          3            ENGINEER/OPERATOR  Houston Fire Department (HFD)   
4          4                  ELECTRICIAN    General Services Department   

   BASE_SALARY             RACE EMPLOYMENT_TYPE  GENDER EMPLOYMENT_STATUS  \
0     121862.0  Hispanic/Latino       Full Time  Female            Active   
1      26125.0  Hispanic/Latino       Full Time  Female            Active   
2      45279.0            White       Full Time    Male            Active   
3      63166.0            White       Full Time    Male            Active   
4      56347.0            White       Full Time    Male            Active   

    HIRE_DATE    JOB_DATE  Normalized_Salary  
0  2006-06-12  2012-10-13        

Custom Function on Movies

In [8]:
def classify_duration(duration):
    if duration < 60:
        return 'Short'
    elif 60 <= duration <= 120:
        return 'Medium'
    else:
        return 'Long'

movies_df['Duration_Category'] = movies_df['duration'].apply(classify_duration)
print(movies_df[['movie_title', 'duration', 'Duration_Category']].head())

                                  movie_title  duration Duration_Category
0                                      Avatar     178.0              Long
1    Pirates of the Caribbean: At World's End     169.0              Long
2                                     Spectre     148.0              Long
3                       The Dark Knight Rises     164.0              Long
4  Star Wars: Episode VII - The Force Awakens       NaN              Long


Pipeline on Titanic

In [11]:
survived_passengers = titanic_df[titanic_df['Survived'] == 1].copy() 
pipeline = Pipeline([
    ('fill_missing_age', SimpleImputer(strategy='mean')),
])
survived_passengers.loc[:, 'Age'] = pipeline.fit_transform(survived_passengers[['Age']])
survived_passengers.loc[:, 'Fare_Per_Age'] = survived_passengers['Fare'] / survived_passengers['Age']

print(survived_passengers[['PassengerId', 'Survived', 'Age', 'Fare', 'Fare_Per_Age']].head())

   PassengerId  Survived   Age     Fare  Fare_Per_Age
1            2         1  38.0  71.2833      1.875876
2            3         1  26.0   7.9250      0.304808
3            4         1  35.0  53.1000      1.517143
8            9         1  27.0  11.1333      0.412344
9           10         1  14.0  30.0708      2.147914
